In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
# Load environment variables in a file called .env
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

# Check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
class FinvizWebsite():
    """
    Create this Website object from the given url using the BeautifulSoup library
    """
    
    def __init__(self, ticker):
        self.ticker = ticker.upper()
        self.url = f"https://finviz.com/quote.ashx?t={self.ticker}&p=d&ty=ea"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
        }
        response = requests.get(self.url, headers=self.headers)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        self.table = soup.find("table", class_="snapshot-table2")     

In [5]:
def messages_for(website):
    system_prompt = """
        You are a financial analysis assistant that analyzes the contents of HTML formated table.
    and provides a summary of the stock's analysis with clear and professional language appropriate for financial research 
    with bulleted important list of **pros** and **cons** , ignoring text that might be navigation related. Repond in markdown.
    """
    
    user_prompt = f"""
        You are looking at a website titled {website.title}.\n
        The contents of this website is as follows; please provide a summary of the stock's analysis from this website in markdown.\n\n
        {website.table}
    """
    
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]


In [6]:
def display_summary(ticker):
    website = FinvizWebsite(ticker)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    summary = response.choices[0].message.content
    display(Markdown(summary))

In [7]:
display_summary("aapl")

# AAPL - Apple Inc Earnings Analysis Summary

Apple Inc (AAPL) exhibits a strong market presence and substantial financial metrics, yet some concerns regarding growth and valuation persist. Below is a concise analysis outlining the key advantages and disadvantages of the stock.

## Key Financial Metrics
- **Current Price**: $201.08
- **Market Capitalization**: $3,003.30B
- **P/E Ratio**: 31.38
- **Forward P/E**: 25.88
- **EPS (ttm)**: 6.41
- **EPS Estimated Next Y**: 7.77
- **Dividend Yield**: 0.51% (Estimated Dividend: $1.03)

## Pros
- **Strong Market Leadership**: Apple is a major constituent of key indices such as DJIA, NDX, and S&P 500.
- **Solid Revenue Generation**: With a revenue of $400.37B and a robust income of $97.29B reported TTM, the company shows a strong financial backbone.
- **High Return on Equity (ROE)**: At 138.02%, indicating effective use of shareholders' equity.
- **Competitive Profit Margins**: The profit margin stands at 24.30%, showcasing effective cost management.
- **Historical Performance**: The stock has shown impressive growth over the long term, with a 10-year performance of 535.32%.

## Cons
- **High Valuation Metrics**: A relatively high P/E ratio of 31.38 could indicate overvaluation compared to historical averages.
- **Weak Recent Performance**: The stock’s performance over the past quarter and year has shown declines of -10.17% and -19.70% respectively, raising concerns for current investors.
- **Insider Transactions**: A decrease in insider ownership by -1.28% could suggest lack of confidence from executives.
- **High PEG Ratio**: With a PEG ratio of 4.09, this value suggests that the stock may be overvalued relative to its earnings growth potential.
- **Debt Levels**: High total debt-to-equity ratio of 1.47 may concern investors regarding long-term solvency.

## Conclusion
While Apple Inc demonstrates strong financial health and potential for long-term growth, investors should consider the high valuation and recent stock performance as factors warranting caution.

In [8]:
display_summary("tsla")

# Tesla Inc. (TSLA) Stock Analysis Summary

## Overview
Tesla Inc. (TSLA) is a high-profile company in the electric vehicle sector, and its stock has gained significant attention for its price volatility and growth potential. Below is a detailed analysis of the stock's financial metrics, performance indicators, and overall sentiment.

## Key Financial Metrics
- **Market Cap**: $1,040.96 billion
- **P/E Ratio (ttm)**: 177.98
- **Diluted EPS (ttm)**: 1.82
- **Forward P/E**: 110.64
- **EPS Estimate Next Year**: 2.93
- **Insider Ownership**: 12.91%
- **Institutional Ownership**: 49.22%
- **Shares Outstanding**: 3.22 billion
- **Cash per Share**: 11.63

## Performance Metrics
- **Performance (Week)**: +0.46%
- **Performance (Month)**: -9.32%
- **Performance (Quarter)**: +18.49%
- **Performance (Year-to-Date)**: -19.86%
- **Performance (Year)**: +72.74%
- **52-Week High/Low**: $488.54 / $182.00
- **Volume**: 88,602,833 shares

## Analysts' Sentiment
- **Mean Recommendation**: 2.59 (Buy/Sell scale)
- **Mean Target Price**: $311.12 

## Pros and Cons

### Pros
- **Strong Annual Growth**: Performance shows a remarkable increase of 72.74% over the past year.
- **Institutional Support**: 49.22% of institutional ownership indicates strong backing by large investors.
- **Significant Revenue**: The company reported total sales of $95.72 billion, demonstrating its strong market presence.
- **Positive EPS Growth Expected**: An estimated EPS growth of 53.25% for the next year suggests strong growth potential.

### Cons
- **High Valuation Metrics**: The P/E ratio of 177.98 indicates the stock might be overvalued compared to current earnings.
- **Negative Performance Trends**: A -19.86% YTD performance suggests challenges in maintaining upward momentum.
- **Quarterly Earnings Decline**: EPS growth is negative at -53.71% year-over-year.
- **Volatility**: With a beta of 2.43, the stock exhibits significant volatility, posing risks for investors.

## Conclusion
Tesla Inc. remains a pivotal player in the electric vehicle industry, marked by high growth potential and significant institutional investment. However, the stock's current high valuation and recent performance struggles present risks that investors should consider before making decisions. The balance between growth prospects and valuation metrics will be crucial for TSLA in the coming quarters.